ABIDA Youssef (p2024398)
VADUREL Benjamin (2000154)

# Projet GNN - Classification de noeuds

# Imports

In [74]:
import networkx as nx
import torch
from torch_geometric.utils import from_networkx


from GAT_Forward1 import GAT_Forward1
from GAT_Forward2 import GAT_Forward2
from GAT_Forward3 import GAT_Forward3
from GAT_Forward4 import GAT_Forward4
from GAT_Pooling import GAT_Pooling
from GAT_MultiHead1 import GAT_MultiHead1
from GAT_Multihead2 import GAT_Multihead2
from GAT_MultiHead_Pooling import GAT_MultiHead_Pooling

from GATBayesianOptimizer import GATBayesianOptimizer
from MultiHeadGATBayesianOptimizer import MultiHeadGATBayesianOptimizer

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import precision_score, root_mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
import numpy as np

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore


In [75]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [76]:
G = nx.read_graphml('../data/airportsAndCoordAndPop.graphml.xml')
G.nodes(data=True)

NodeDataView({'0': {'lon': -145.50972222222222, 'lat': -17.35388888888889, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Anaa'}, '1': {'lon': -140.95, 'lat': -18.066666666666666, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Hao Island'}, '2': {'lon': -149.6, 'lat': -17.55, 'population': 26357, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Papeete'}, '3': {'lon': -135.0, 'lat': -23.033333333333335, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Gambier Island'}, '4': {'lon': -143.65725, 'lat': -16.584888888888887, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Makemo'}, '5': {'lon': 174.79166666666666, 'lat': -37.00805555555556, 'population': 417910, 'country': 'NEW_ZEALAND', 'city_name': 'Auckland'}, '6': {'lon': -139.03333333333333, 'lat': -9.8, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 'city_name': 'Atuona'}, '7': {'lon': -151.75, 'lat': -16.45, 'population': 10000, 'country': 'FRENCH_POLYNESIA', 

In [77]:
# lets make an occurence list of each node degree
occurence = {}
for node in G.nodes():
    degree = G.degree(node)
    if degree not in occurence:
        occurence[degree] = 1
    else:
        occurence[degree] += 1
    
# sort the occurence list
occurence = dict(sorted(occurence.items(), key=lambda item: item[0]))

print(occurence)



{1: 725, 2: 869, 3: 445, 4: 261, 5: 182, 6: 135, 7: 74, 8: 50, 9: 43, 10: 55, 11: 35, 12: 28, 13: 24, 14: 26, 15: 28, 16: 22, 17: 34, 18: 12, 19: 15, 20: 17, 21: 17, 22: 7, 23: 16, 24: 7, 25: 9, 26: 13, 27: 8, 28: 9, 29: 9, 30: 5, 31: 3, 32: 4, 33: 8, 34: 4, 35: 12, 36: 3, 37: 4, 38: 7, 39: 3, 40: 2, 41: 4, 42: 3, 43: 1, 44: 3, 45: 3, 46: 3, 47: 4, 48: 1, 49: 5, 50: 6, 51: 4, 53: 5, 54: 1, 55: 2, 56: 3, 57: 1, 59: 6, 61: 2, 62: 1, 63: 3, 64: 3, 65: 1, 66: 1, 68: 2, 69: 2, 75: 3, 76: 1, 77: 3, 78: 2, 79: 2, 80: 2, 81: 1, 83: 1, 84: 1, 85: 2, 86: 1, 87: 2, 91: 1, 93: 1, 94: 1, 97: 2, 98: 2, 99: 1, 100: 4, 104: 1, 107: 1, 108: 1, 109: 2, 110: 1, 111: 2, 112: 1, 113: 1, 115: 3, 119: 1, 129: 1, 132: 1, 134: 1, 135: 1, 142: 1, 143: 1, 144: 1, 147: 1, 171: 1, 178: 1, 179: 1, 183: 1, 191: 1, 234: 1, 240: 1, 248: 1}


# Data description
 - lon : float
 - lat : float
 - population : int
 - country : string (exemple : 'FRENCH_POLYNESIA')
 - city_name : string (exemple : 'Anaa')

In [78]:
def encode_city_name(graph):
    city_names = nx.get_node_attributes(graph, 'city_name')
    print(city_names)

    encoder = LabelEncoder()
    print(city_names.values())
    city_names_encoder = encoder.fit(list(city_names.values()))

    for node in graph.nodes():
        graph.nodes[node]['city_name'] = city_names_encoder.transform([graph.nodes[node]['city_name']])[0]

    graph.nodes(data=True)

In [48]:
# nx.draw(graph, with_labels=False, node_size=8)

In [82]:
encode_city_name(G)

{'0': 'Anaa', '1': 'Hao Island', '2': 'Papeete', '3': 'Gambier Island', '4': 'Makemo', '5': 'Auckland', '6': 'Atuona', '7': 'Bora Bora', '8': 'Honolulu', '9': 'Huahine', '10': 'Easter Island', '11': 'Kaukura Atoll', '12': 'Los Angeles', '13': 'Moorea', '14': 'Nuku Hiva', '15': 'Noumea', '16': 'Osaka', '17': 'Mataiva', '18': 'Rarotonga', '19': 'Raiatea Island', '20': 'Rangiroa Island', '21': 'Rurutu', '22': 'Tikehau Atoll', '23': 'Takaroa', '24': 'Tubuai', '25': 'Tokyo', '26': 'Wallis Island', '27': 'Manihi', '28': 'Apia', '29': 'Blenheim', '30': 'Brisbane', '31': 'Buenos Aires', '32': 'Christchurch', '33': 'Cairns', '34': 'Denpasar Bali', '35': 'Gisborne', '36': 'Hong Kong', '37': 'Hamilton', '38': 'Niue', '39': 'Kaitaia', '40': 'Kerikeri', '41': 'Kuala Lumpur', '42': 'Melbourne', '43': 'Nadi', '44': 'Nagoya', '45': 'Norfolk Island', '46': 'Napier-Hastings', '47': 'New Plymouth', '48': 'Nelson', '49': 'Perth', '50': 'Palmerston North', '51': 'Rotorua', '52': 'Seoul', '53': 'Singapore',

In [79]:
def remove_isolated_nodes(min_degree=1):
    graph = nx.read_graphml('../data/airportsAndCoordAndPop.graphml.xml')
    nodes_to_remove = [node for node in graph.nodes if graph.degree(node) < min_degree]
    graph.remove_nodes_from(nodes_to_remove)
    return graph

# new_graph = remove_isolated_nodes(min_degree=3)

# nx.draw(new_graph, with_labels=False, node_size=8) 
# encode_city_name(new_graph)

In [80]:
def convert_graph_to_data(graph):
    data = from_networkx(graph, group_node_attrs=["lon", "lat", "city_name"])
    
    return data

def print_data_info(data):
    print(data)
    print(data.num_nodes)
    print(data.num_edges)
    print(data.num_node_features)

In [83]:
# data = convert_graph_to_data(new_graph)
data = convert_graph_to_data(G)

print_data_info(data)

Data(edge_index=[2, 27094], population=[3363], country=[3363], x=[3363, 3])
3363
27094
3


In [84]:
data_copy = data.clone()

# Exploratory data analysis

In [85]:
def create_df(graph):
    nodes_data = []
    for node, feature in graph.nodes(data=True):
        nodes_data.append(feature)

    df = pd.DataFrame(nodes_data)
    return df

In [86]:
# df = create_df(new_graph)
df = create_df(G)

## Visualisation

In [87]:
def visualize_distribution(data_frame):
    for column in data_frame.columns:
        if data_frame[column].dtype in [np.float64, np.int64]:
            plt.figure(figsize=(10, 6))
            sns.histplot(data_frame[column], kde=True)
            plt.title(f'Distribution of {column}')
            plt.show()

            counts, bin_edges = np.histogram(data_frame[column], bins='auto')
            bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

            print(f"Values for {column}:")
            for bin_center, count in zip(bin_centers, counts):
                print(f"Bin center: {bin_center}, Count: {count}")

In [35]:
# visualize_distribution(df)

> La distribution des valeurs pour la population est très inégale, avec une grande majorité des valeurs concentrées sur les bins inférieurs.

# Dirty data?

In [36]:
def calc_z_scores(data_frame):
    z_scores = np.abs(zscore(data_frame[['lon', 'lat', 'population', 'city_name']]))
    print(f"Z-scores: {z_scores}")
    outliers = np.where(z_scores > 3)
    print(f"Outliers detected at indices: {outliers}")

    # Afficher les valeurs des outliers
    for column in data_frame.columns:
        outlier_indices = outliers[0][outliers[1] == data_frame.columns.get_loc(column)]
        outlier_values = data_frame.loc[outlier_indices, column]
        print(f"Outliers in {column}: {outlier_values.values}")


    # for column in data_frame.columns:
    #     if data_frame[column].dtype in [np.float64, np.int64]:
    #         plt.figure(figsize=(10, 6))
    #         sns.boxplot(x=data_frame[column])
    #         plt.title(f'Boxplot of {column}')
    #         plt.show()

    return z_scores, outliers


In [ ]:
z_scores, outliers = calc_z_scores(df)

> Les outliers détectés dans la colonne population peuvent être des anomalies ou des données correctes mais extrêmes. Les données de la population seront donc retirées pour netttoyer les données.

In [ ]:
df = df.drop(columns=['population'])
data_copy.population = None 

print(df.shape)
print(data_copy)

# Feature engineering

Pour améliorer la performance de nos modèles, nous avons créé une nouvelle caractéristique à partir des données existantes. La caractéristique de densité de population en divisant la population par la longitude.

In [19]:
# df['population_density'] = df['population'] / df['lon']

Pour identifier les caractéristiques les plus importantes pour la prédiction du pays (`country`), nous avons utilisé une forêt aléatoire. Cette méthode permet de mesurer l'importance relative de chaque caractéristique.

Ces résultats montrent que la longitude (`lon`) est la caractéristique la plus importante pour prédire le pays, suivie de la latitude (`lat`) et de la population (`population`).

In [88]:
print(f"data_copy.x shape: {data_copy.x.shape}, data_copy.edge_index shape: {data_copy.edge_index.shape}")
print(f"Min index in edge_index: {data_copy.edge_index.min()}, Max index: {data_copy.edge_index.max()}, Number of nodes: {data_copy.x.shape[0]}")

data_copy.x shape: torch.Size([3363, 3]), data_copy.edge_index shape: torch.Size([2, 27094])
Min index in edge_index: 0, Max index: 3362, Number of nodes: 3363


# Nettoyer et centrer réduire les données

In [ ]:
# Suppression des outliers
mask_cleaned = (np.abs(z_scores) < 3).all(axis=1)
df_cleaned = df[mask_cleaned]

new_indices = np.where(mask_cleaned)[0]
node_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(new_indices)}

updated_edge_index = data_copy.edge_index.clone()
for i in range(updated_edge_index.shape[1]):
    u, v = updated_edge_index[:, i].numpy()
    if u in node_mapping and v in node_mapping:
        updated_edge_index[0, i] = node_mapping[u]
        updated_edge_index[1, i] = node_mapping[v]
    else:
        updated_edge_index[:, i] = -1

valid_edges = (updated_edge_index[0, :] != -1)
updated_edge_index = updated_edge_index[:, valid_edges]

# Mise à jour des données pour le modèle
# scaler = StandardScaler()
# df_scaled = pd.DataFrame(scaler.fit_transform(df_cleaned[['lon', 'lat', 'city_name']]), columns=['lon', 'lat', 'city_name'])

# data_copy.x = torch.tensor(df_scaled.values, dtype=torch.float)
data_copy.x = torch.tensor(df_cleaned[['lon', 'lat', 'city_name']].values, dtype=torch.float)
data_copy.edge_index = updated_edge_index
data_copy.country = df_cleaned['country'].to_list()

# Vérification
print(f"data_copy.x shape: {data_copy.x.shape}, data_copy.edge_index shape: {data_copy.edge_index.shape}")
print(f"Min index in edge_index: {data_copy.edge_index.min()}, Max index: {data_copy.edge_index.max()}, Number of nodes: {data_copy.x.shape[0]}")



In [89]:
print(data_copy)
print(data_copy.x)

Data(edge_index=[2, 27094], population=[3363], country=[3363], x=[3363, 3])
tensor([[-145.5097,  -17.3539,   99.0000],
        [-140.9500,  -18.0667, 1071.0000],
        [-149.6000,  -17.5500, 2207.0000],
        ...,
        [-113.2039,   54.7431, 3199.0000],
        [ 146.6000,   -6.1333, 3237.0000],
        [ 147.2500,   -6.0000, 3161.0000]])


In [164]:
# # Suppression des outliers
# mask_cleaned = (np.abs(z_scores) < 3).all(axis=1)
# df_cleaned = df[mask_cleaned]

# scaler = StandardScaler()
# df_scaled = pd.DataFrame(scaler.fit_transform(df_cleaned[['lon', 'lat', 'city_name']]), columns=['lon', 'lat', 'city_name'])

# # Mise à jour des données pour le modèle
# data_copy.x = torch.tensor(df_scaled.values, dtype=torch.float)
# data_copy.country = df_cleaned['country'].to_list()

# print(data_copy)
# print(df_scaled.shape)
# print(f"data_copy.x shape: {data_copy.x.shape}, data_copy.edge_index shape: {data_copy.edge_index.shape}")
# print(f"Min index in edge_index: {data_copy.edge_index.min()}, Max index: {data_copy.edge_index.max()}, Number of nodes: {data_copy.x.shape[0]}")


# Preprocessing

In [90]:

encoder = LabelEncoder()

country_encoder = encoder.fit(data_copy.country)
data_copy.country = encoder.transform(data_copy.country)

target_tensor = torch.tensor(data_copy.country, dtype=torch.long)
data_copy.y = target_tensor
data_copy.num_classes = len(set(data_copy.country))


train_ratio = 0.8

mask = torch.rand(data_copy.num_nodes) < train_ratio
data_copy.train_mask = mask
data_copy.test_mask = ~mask

temp = torch.zeros((data_copy.num_nodes, data_copy.num_node_features), dtype=torch.float)
temp[data_copy.train_mask] = data_copy.x[data_copy.train_mask]
data_copy.x = temp

data_copy = data_copy.to(device)

print(f"train_mask shape: {data_copy.train_mask.shape}, test_mask shape: {data_copy.test_mask.shape}")
print(f"x shape: {data_copy.x.shape}, num_nodes: {data_copy.num_nodes}")
print(f"y min: {data_copy.y.min()}, y max: {data_copy.y.max()}, num_classes: {data_copy.num_classes}")
print(f"data_copy.x dtype: {data_copy.x.dtype}")
print(f"data_copy.y dtype: {data_copy.y.dtype}")


# print(data_copy.num_classes)
# print(data_copy)

train_mask shape: torch.Size([3363]), test_mask shape: torch.Size([3363])
x shape: torch.Size([3363, 3]), num_nodes: 3363
y min: 0, y max: 211, num_classes: 212
data_copy.x dtype: torch.float32
data_copy.y dtype: torch.int64


In [91]:
def test_model(model, data):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    data = data.to(device)

    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index).argmax(dim=1)

    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())

    precision = precision_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu(), average='weighted')
    rmse = root_mean_squared_error(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())

    return acc, precision, rmse

In [95]:
clfs = {
    'GAT_Forward1': GAT_Forward1,
    'GAT_Forward2': GAT_Forward2,
    'GAT_Forward3': GAT_Forward3,
    'GAT_Forward4': GAT_Forward4
}


def calc_scoring(y_test, y_pred):
  return (accuracy_score(y_test, y_pred) + precision_score(y_test, y_pred)) / 2

def compare_clfs(data, epochs):
    kf = KFold(n_splits=10, shuffle=True, random_state=0)

    max_accuracy = 0
    clf_max_accu = ''
    max_auc = 0
    clf_max_auc = ''
    max_acc_prec = 0
    clf_max_acc_prec = ''

    scorers = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'roc_auc': 'roc_auc',
        'rmse': 'root_mean_squared_error',
        'acc_prec': make_scorer(calc_scoring)
    }

    for i in clfs:
        clf = clfs[i](data.num_node_features, data.num_classes).to(device)

        cross_validate_res = cross_validate(clf, data, cv=kf, scoring=scorers)
        
        mean_accuracy = np.mean(cross_validate_res['test_accuracy'])
        mean_precision = np.mean(cross_validate_res['test_precision'])
        mean_roc_auc = np.mean(cross_validate_res['test_roc_auc'])
        mean_rmse = np.mean(cross_validate_res['test_rmse'])
        mean_acc_prec = np.mean(cross_validate_res['test_acc_prec'])

        std_accuracy = np.std(cross_validate_res['test_accuracy'])
        std_precision = np.std(cross_validate_res['test_precision'])
        std_auc = np.std(cross_validate_res['test_roc_auc'])
        std_rmse = np.std(cross_validate_res['test_rmse'])
        std_acc_prec = np.std(cross_validate_res['test_acc_prec'])

        if mean_accuracy > max_accuracy:
            max_accuracy = mean_accuracy
            clf_max_accu = clf

        if mean_precision > max_acc_prec:
            max_acc_prec = mean_precision
            clf_max_acc_prec = clf

        if mean_roc_auc > max_auc:
            max_auc = mean_roc_auc
            clf_max_auc = clf

        if mean_rmse > max_rmse:
            max_rmse = mean_rmse
            clf_max_rmse = clf

        if mean_acc_prec > max_acc_prec:
            max_acc_prec = mean_acc_prec
            clf_max_acc_prec = clf

        print("Accuracy for {0} is: {1:.3f} +/- {2:.3f}".format(i, mean_accuracy, std_accuracy))
        print("Precision for {0} is: {1:.3f} +/- {2:.3f}".format(i, mean_precision, std_precision))
        print("AUC for {0} is: {1:.3f} +/- {2:.3f}".format(i, mean_roc_auc, std_auc))
        print("RMSE for {0} is: {1:.3f} +/- {2:.3f}".format(i, mean_rmse, std_rmse))
        print("Accuracy + Precision / 2 for {0} is: {1:.3f} +/- {2:.3f}".format(i, mean_acc_prec, std_acc_prec))

    print("Meilleur model pour l'accuarcy: ", clf_max_accu, " avec ", max_accuracy)
    print("Meilleur model pour la precision: ", clf_max_acc_prec, " avec ", max_acc_prec)
    print("Meilleur model pour l'AUC: ", clf_max_auc, " avec ", max_auc)
    print("Meilleur model pour le RMSE: ", clf_max_rmse, " avec ", max_rmse)
    print("Meilleur model pour l'accuracy + precision: ", clf_max_acc_prec, " avec ", max_acc_prec)

# compare_clfs(data_copy, 1000)

# Forward Models 1, 2, 3

In [22]:
models = {
    'GAT_Forward1': GAT_Forward1(data_copy.x.shape[1], 8, data_copy.num_classes),
    'GAT_Forward2': GAT_Forward2(data_copy.x.shape[1], 8, data_copy.num_classes),
    'GAT_Forward3': GAT_Forward3(data_copy.x.shape[1], 8, data_copy.num_classes),
    'GAT_Forward4': GAT_Forward4(data_copy.x.shape[1], 8, data_copy.num_classes)
}

def run_test_models(data, epochs, models):
    best_model = None

    for model_name, model in models.items():
        model = model.to(device)
        model.fit(data, epochs)

        test = model.test_model(data)

        if best_model is None or test[0] > best_model[1][0]:
            best_model = (model_name, test)

        print(f"Pour le modele {model_name} : {model.test_model(data)}")

    return best_model

# best_model = run_test_models(data_copy, 7000, models)
# print(f"Meilleur modele : {best_model[0]} avec {best_model[1]}")

# Bayesian Optimization

In [96]:
def run_bayesian_optim(data, nb_trials, train_nbr_epochs):
    bayesian_optimizer = GATBayesianOptimizer(data, nb_trials, train_nbr_epochs)

    forward1_study = bayesian_optimizer.create_study(model_class=GAT_Forward1)
    bayesian_optimizer.get_best_study(forward1_study)
    print("Forward 1 done")
    print("\n")

    forward2_study = bayesian_optimizer.create_study(model_class=GAT_Forward2)
    bayesian_optimizer.get_best_study(forward2_study)
    print("Forward 2 done")
    print("\n")

    forward3_study = bayesian_optimizer.create_study(model_class=GAT_Forward3)
    bayesian_optimizer.get_best_study(forward3_study)
    print("Forward 3 done")
    print("\n")

    forward4_study = bayesian_optimizer.create_study(model_class=GAT_Forward4)
    bayesian_optimizer.get_best_study(forward4_study)
    print("Forward 4 done")
    print("\n")

    # pooling_study = bayesian_optimizer.create_study(model_class=GAT_Pooling)
    # bayesian_optimizer.get_best_study(pooling_study)
    # print("Pooling done")
    # print("\n")


In [92]:
def run_bayesian_optim(data, nb_trials, train_nbr_epochs, model_class):
    bayesian_optimizer = GATBayesianOptimizer(data, nb_trials, train_nbr_epochs)
    study = bayesian_optimizer.create_study(model_class=model_class)
    best_accuracy, best_precision, best_rmse, best_hyperparameters = bayesian_optimizer.get_best_study(study)

    print("\n")
    print(f"Best accuracy: {best_accuracy}")
    print(f"Best precision: {best_precision}")
    print(f"Best RMSE: {best_rmse}")
    print(f"Best hyperparameters: {best_hyperparameters}")

    return best_accuracy, best_precision, best_rmse, best_hyperparameters

In [98]:
def update_saved_model(save_path, best_acc, best_prec, best_rmse, best_hyperparam, best_model, data):
    try:
        checkpoint = torch.load(save_path)
        saved_best_acc = checkpoint['best_accuracy']
        saved_best_prec = checkpoint['best_precision']
        saved_best_rmse = checkpoint['best_rmse']
    except FileNotFoundError:
        saved_best_acc = -float('inf')
        saved_best_prec = -float('inf')
        saved_best_rmse = float('inf')

    if (best_acc > saved_best_acc) and (best_prec > saved_best_prec) and (best_rmse < saved_best_rmse):
        best_model.save_model(
            save_path,
            best_accuracy=best_acc,
            best_precision=best_prec,
            best_rmse=best_rmse,
            best_hyperparameters=best_hyperparam,
            input_dim=data.x.shape[1],
            output_dim=data.num_classes
        )
        print(f"Model saved with improved results: Acc={best_acc}, Prec={best_prec}, RMSE={best_rmse}")
    else:
        print(f"Model not saved. Current results are not better than saved results: Acc={saved_best_acc}, Prec={saved_best_prec}, RMSE={saved_best_rmse}")


In [94]:
def train_and_save_model(model_class, data, save_path, train_nbr_epochs, best_acc, best_prec, best_rmse, best_hyperparam):
    best_model = model_class(
        data.x.shape[1],
        best_hyperparam['nb_neurons'],
        data.num_classes,
        dropout=best_hyperparam['dropout']
    ).to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    best_model.fit(data, epochs=train_nbr_epochs, lr=best_hyperparam['lr'], wd=best_hyperparam['weight_decay'])

    update_saved_model(save_path, best_acc, best_prec, best_rmse, best_hyperparam, best_model, data)


In [100]:
best_acc1, best_prec1, best_rmse1, best_hyper1 = run_bayesian_optim(data_copy, 15, 7000, GAT_Forward1)

[I 2024-10-20 00:43:26,246] A new study created in memory with name: no-name-983ee254-bfae-4959-ab65-670dd2bc045e
[I 2024-10-20 00:43:26,450] Trial 0 finished with values: [0.10260336906584992, 0.04233285856563039, 88.49279912909813] and parameters: {'lr': 0.019709605016780804, 'weight_decay': 2.14583434208804e-05, 'nb_neurons': 449, 'dropout': 0.038473443835620624}.
[I 2024-10-20 00:43:26,472] Trial 1 finished with values: [0.0, 0.0, 111.29869583117893] and parameters: {'lr': 0.007223147566811225, 'weight_decay': 1.3846926768733563e-05, 'nb_neurons': 106, 'dropout': 0.36627299198224295}.
[I 2024-10-20 00:43:26,499] Trial 2 finished with values: [0.11944869831546708, 0.030417808332489218, 106.79118322549562] and parameters: {'lr': 0.014116811888783049, 'weight_decay': 1.6013908486923576e-05, 'nb_neurons': 345, 'dropout': 0.33905537144027575}.
[I 2024-10-20 00:43:26,540] Trial 3 finished with values: [0.0030627871362940277, 0.1807044410413476, 116.08092440500185] and parameters: {'lr': 



Best accuracy: 0.07656967840735068
Best precision: 0.04294553498228843
Best RMSE: 99.11149691547874
Best hyperparameters: {'lr': 0.006779931123545761, 'weight_decay': 3.478488449736036e-05, 'nb_neurons': 449, 'dropout': 0.21617869164173031}


In [ ]:
best_acc2, best_prec2, best_rmse2, best_hyper2 = run_bayesian_optim(data_copy, 4, 7000, GAT_Forward2)

In [ ]:
best_acc3, best_prec3, best_rmse3, best_hyper3 = run_bayesian_optim(data_copy, 4, 7000, GAT_Forward3)

In [ ]:
best_acc4, best_prec4, best_rmse4, best_hyper4 = run_bayesian_optim(data_copy, 4, 7000, GAT_Forward4)

# Saving models

In [ ]:
train_and_save_model(GAT_Forward1, data_copy, '../runs/avant_analyse/models/GAT_Forward1.pth', 3, best_acc1, best_prec1, best_rmse1, best_hyper1)

In [ ]:
train_and_save_model(GAT_Forward2, data_copy, '../runs/avant_analyse/models/GAT_Forward2.pth', 3, best_acc2, best_prec2, best_rmse2, best_hyper2)

In [ ]:
train_and_save_model(GAT_Forward3, data_copy, '../runs/avant_analyse/models/GAT_Forward3.pth', 3, best_acc3, best_prec3, best_rmse3, best_hyper3)

In [ ]:

train_and_save_model(GAT_Forward4, data_copy, '../runs/avant_analyse/models/GAT_Forward4.pth', 3, best_acc4, best_prec4, best_rmse4, best_hyper4)

# Load models

In [99]:
b_model1, hyper1, checkpoint1 = GAT_Forward1.load_model('../runs/avant_analyse/models/GAT_Forward1.pth')
b_model1 = b_model1.to(device)
accuracy, precision, rmse = b_model1.test_model(data_copy)
print(f"Accuracy: {accuracy}, Precision: {precision}, RMSE: {rmse}")

Accuracy: 0.7825421133231241, Precision: 0.763689633976716, RMSE: 49.090581026123424


c:\Users\Josephir\Documents\Josephir\UCBL\M1+\BioInspiredMachineLearning\TP-Cazabet\bioml-gnn-classification\GAT\GAT_Forward1.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featu

In [ ]:
b_model2, hyper2, checkpoint2 = GAT_Forward2.load_model('../runs/avant_analyse/models/GAT_Forward2.pth')

In [ ]:
b_model3, hyper3, checkpoint3 = GAT_Forward3.load_model('../runs/avant_analyse/models/GAT_Forward3.pth')

In [ ]:

b_model4, hyper4, checkpoint4 = GAT_Forward4.load_model('../runs/avant_analyse/models/GAT_Forward4.pth')

WINNER : `Forward1`

Best Accuracy: `0.7093718291054687`

Best hyperparameters: `{'lr': 0.0010312531082429088, 'weight_decay': 2.7069760917328855e-05, 'nb_neurons': 305, 'dropout': 0.0015058484884345191}`

optim : `Adam`

avec 7000 epochs

# Testing hyperparams on the other models

In [95]:
def multi_runs(model_class, data, nb_runs, device, save_path, dropout=0.0015058484884345191, lr=0.0010312531082429088, wd=2.7069760917328855e-05):
    all_runs = {}

    hyper = {
        'lr': lr,
        'weight_decay': wd,
        'nb_neurons': 305,
        'dropout': dropout
    }

    for run in range(nb_runs):
        print(f"Run {run}")
        current_model = model_class(data.x.shape[1], 305, data.num_classes, dropout=dropout)
        current_model = current_model.to(device)
        current_model.fit(data, 7000, lr=lr, wd=wd)
        accuracy, precision, rmse = current_model.test_model(data)

        update_saved_model(save_path, accuracy, precision, rmse, hyper, current_model, data)
        
        all_runs[run] = (accuracy, precision, rmse)
    
    print("\n")
    for run in all_runs:
        print(f"Run {run}: Accuracy: {all_runs[run][0]}, Precision: {all_runs[run][1]}, RMSE: {all_runs[run][2]}")

    avg_accuracy = sum([all_runs[run][0] for run in all_runs]) / nb_runs
    avg_precision = sum([all_runs[run][1] for run in all_runs]) / nb_runs
    avg_rmse = sum([all_runs[run][2] for run in all_runs]) / nb_runs

    return avg_accuracy, avg_precision, avg_rmse

In [96]:
gat_f1_avg_acc, gat_f1_avg_prec, gat_f1_avg_rmse = multi_runs(GAT_Forward1, data_copy, 12, device, '../runs/avant_analyse/models/GAT_Forward1.pth')
print(f"\n Average Accuracy: {gat_f1_avg_acc}, Average Precision: {gat_f1_avg_prec}, Average RMSE: {gat_f1_avg_rmse}")

Run 0


C:\Users\Josephir\AppData\Local\Temp\ipykernel_2876\2979317545.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path)


Model saved with improved results: Acc=0.776416539050536, Prec=0.775580774749539, RMSE=42.37230162346425
Run 1


C:\Users\Josephir\AppData\Local\Temp\ipykernel_2876\2979317545.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path)


Model not saved. Current results are not better than saved results: Acc=0.776416539050536, Prec=0.775580774749539, RMSE=42.37230162346425
Run 2


...

C:\Users\Josephir\AppData\Local\Temp\ipykernel_2876\2979317545.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path)


Model not saved. Current results are not better than saved results: Acc=0.776416539050536, Prec=0.775580774749539, RMSE=42.37230162346425
Run 10


C:\Users\Josephir\AppData\Local\Temp\ipykernel_2876\2979317545.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path)


Model not saved. Current results are not better than saved results: Acc=0.776416539050536, Prec=0.775580774749539, RMSE=42.37230162346425
Run 11
Model saved with improved results: Acc=0.7825421133231241, Prec=0.763689633976716, RMSE=49.090581026123424


Run 0 : Accuracy: 0.776416539050536, Precision: 0.775580774749539, RMSE: 42.37230162346425
Run 1 : Accuracy: 0.7733537519142419, Precision: 0.7726124251364681, RMSE: 47.12748667179272
Run 2 : Accuracy: 0.6493108728943339, Precision: 0.6514244930208488, RMSE: 53.88708346315832
Run 3 : Accuracy: 0.7549770290964778, Precision: 0.7485375242758453, RMSE: 49.14899060186522
Run 4 : Accuracy: 0.6891271056661562, Precision: 0.7053428257217839, RMSE: 51.2880344803501
Run 5 : Accuracy: 0.7641653905053599, Precision: 0.7678638330306743, RMSE: 45.88541891733134
Run 6 : Accuracy: 0.7534456355283308, Precision: 0.746245850046124, RMSE: 48.63167503009507
Run 7 : Accuracy: 0.7626339969372129, Precision: 0.7553650588953241, RMSE: 49.98211012264693
Run 8 

C:\Users\Josephir\AppData\Local\Temp\ipykernel_2876\2979317545.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(save_path)


In [ ]:
gat_f2_avg_acc, gat_f2_avg_prec, gat_f2_avg_rmse = multi_runs(GAT_Forward2, data_copy, 1, device, '../runs/avant_analyse/models/GAT_Forward2.pth')

In [ ]:
gat_f3_avg_acc, gat_f3_avg_prec, gat_f3_avg_rmse = multi_runs(GAT_Forward3, data_copy, 1, device, '../runs/avant_analyse/models/GAT_Forward3.pth')

In [ ]:
gat_f4_avg_acc, gat_f4_avg_prec, gat_f4_avg_rmse = multi_runs(GAT_Forward4, data_copy, 1, device, '../runs/avant_analyse/models/GAT_Forward4.pth')

In [ ]:
# gat_pooling_avg_acc, gat_pooling_avg_prec, gat_pooling_avg_rmse = multi_runs(GAT_Pooling, data_copy, 10, device)

# Updating saved models

In [17]:
def run_bayesian_optim_multi(data, nb_trials, train_nbr_epochs):
    bayesian_optimizer = MultiHeadGATBayesianOptimizer(data, nb_trials, train_nbr_epochs)

    multi1_study = bayesian_optimizer.create_study(model_class=GAT_MultiHead1)
    bayesian_optimizer.get_best_study(multi1_study)
    print("GAT_MultiHead1 1 done")
    print("\n")

    multi2_study = bayesian_optimizer.create_study(model_class=GAT_Multihead2)
    bayesian_optimizer.get_best_study(multi2_study)
    print("GAT_MultiHead1 2 done")
    print("\n")

    # multi_pooling_study = bayesian_optimizer.create_study(model_class=GAT_MultiHead_Pooling)
    # bayesian_optimizer.get_best_study(multi_pooling_study)
    # print("GAT_MultiHead1 Pooling done")
    # print("\n")

# need to do graphs : 
- acc en fonction de l'epoch, pour base et multi head et pooling
- acc et loss
- euu jsp quoi d autre


In [ ]:
run_bayesian_optim_multi(data_copy, 12, 7000)

In [ ]:
b_model1, hyper1, checkpoint = GAT_Forward1.load_model('../runs/avant_analyse/models/GAT_Forward1.pth')

In [ ]:
b_model1 = b_model1.to(device)
# b_model1.forward(data_copy.x, data_copy.edge_index)
print(b_model1.test_model(data_copy))

In [ ]:
accuracy, precision, rmse = b_model1.test_model(data_copy)
print(f"Accuracy: {accuracy}, Precision: {precision}, RMSE: {rmse}")

# Testing save and load model

In [ ]:
new_model = GAT_Forward1(data_copy.x.shape[1], 2, data_copy.num_classes, dropout=0.5)
new_model = new_model.to(device)
new_model.fit(data_copy, 100)
new_model.test_model(data_copy)

print(new_model.test_model(data_copy))

new_model.save_model("../runs/avant_analyse/models/GAT_Forward1_test.pth", 0.5, 0.5, 0.5, {'lr': 0.001, 'weight_decay': 0.0001, 'nb_neurons': 2, 'dropout': 0.5}, data_copy.x.shape[1], data_copy.num_classes)

In [ ]:
test, _, _  = GAT_Forward1.load_model("../runs/avant_analyse/models/GAT_Forward1_test.pth")
test = test.to(device)
print(test.test_model(data_copy))